In [1]:
import mlflow

In [ ]:
# log system metrics every 10 seconds
mlflow.enable_system_metrics_logging()

In [3]:
# Set the tracking URI and experiment for subsequent runs

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Smart_Recycling")

# set up run name

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1758209989740, experiment_id='1', last_update_time=1758209989740, lifecycle_stage='active', name='Smart_Recycling', tags={'dataset': 'garbage-dataset-v1',
 'framework': 'tensorflow-keras',
 'mlflow.experimentKind': 'custom_model_development',
 'mlflow.note.content': ' This experiment focuses on Smart Recycling using '
                        'computer vision. \n'
                        '    The goal is to classify waste items into 8 '
                        'categories: battery, biological, clothes, glass, '
                        'metal, paper, plastic, and trash. \n'
                        '    Different model architectures and augmentation '
                        'strategies are tested to evaluate their performance '
                        'and identify the best approach \n'
                        '    for robust and scalable recycling classification.',
 'num_classes': '8',
 'project_name': 'smart-recycling'

In [4]:
# Configure TensorFlow autologging with MLflow, disabling dataset logging to avoid large data uploads
# for first experimentation runs. Model logging is also disabled to manage storage.


mlflow.tensorflow.autolog(log_datasets=False, log_models=False)

# Set a custom run name for better identification in the MLflow UI
mlflow.set_tag("mlflow.runName", "baseline")

2025-09-18 18:26:34.870835: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-18 18:26:34.906699: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-18 18:26:35.731824: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [6]:
from smart_r.utils import get_tensorflow_dataset

# Tensorflow Dataset loading

train_dataset = get_tensorflow_dataset(
    image_folder="/home/ubuntu/dev/smart_r/garbage-dataset-v1/train",
    image_size=(224, 224),
    batch_size=64,
    label_mode="categorical",
    shuffle=True,  # shuffle True for training dataset
    seed=42,
)

val_dataset = get_tensorflow_dataset(
    image_folder="/home/ubuntu/dev/smart_r/garbage-dataset-v1/val",
    image_size=(224, 224),
    batch_size=64,
    label_mode="categorical",
    shuffle=False,  # shuffle False for validation dataset
    seed=42,
)

test_dataset = get_tensorflow_dataset(
    image_folder="/home/ubuntu/dev/smart_r/garbage-dataset-v1/test",
    image_size=(224, 224),
    batch_size=64,
    label_mode="categorical",
    shuffle=False,  # shuffle needs to be false for later evaluation
    seed=42,
)

Found 14819 files belonging to 8 classes.


I0000 00:00:1758212796.885857  199453 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9284 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 1973 files belonging to 8 classes.
Found 2970 files belonging to 8 classes.


In [7]:
import tensorflow as tf

# setup mixed precision if wanted
# Reference: https://www.tensorflow.org/guide/mixed_precision
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Section for Model Creation

base_model = tf.keras.applications.MobileNetV3Large(
    include_top=False,
    input_shape=(224, 224, 3),
    weights="imagenet",
    pooling=None,
    include_preprocessing=True,
)

base_model.trainable = (
    False  # Freeze the base model, only the top layers will be trained
)

model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(224, 224, 3)),
        tf.keras.layers.RandomFlip("horizontal_and_vertical", seed=42),
        tf.keras.layers.RandomRotation(0.12, seed=42),
        tf.keras.layers.RandomZoom(0.12, seed=42),
        tf.keras.layers.RandomContrast(0.12, seed=42),
        tf.keras.layers.RandomBrightness(0.12, seed=42),
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(8, activation="softmax", dtype="float32"),
    ]
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.F1Score(average="weighted")],
)

In [8]:
from smart_r.utils import compute_class_weights
import numpy as np

# Compute the class weights, thats important when your dealing with an imbalanced dataset

y_true = np.concatenate([y for x, y in train_dataset], axis=0)
class_weight = compute_class_weights(np.argmax(y_true, axis=1), class_weight="balanced")

class_weight

2025-09-18 18:26:41.347568: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


{0: np.float64(2.616348870056497),
 1: np.float64(2.47975234270415),
 2: np.float64(0.3381480467323841),
 3: np.float64(0.8071350762527233),
 4: np.float64(2.4214052287581698),
 5: np.float64(0.7048611111111112),
 6: np.float64(1.2448756720430108),
 7: np.float64(2.608978873239437)}

In [9]:
# Configure Training Callbacks


# stop training when the validation loss is not decreasing after 10 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
)


# This is not strictly necessary with the adam optimizer, but can help in some cases
# it will reduce the learning rate by a factor of 0.1 if the validation loss is not decreasing after 5 epochs to a minimum of 1e-7
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=5,
    min_lr=1e-7,
)

In [10]:
# Actual train the model


# epochs can be set to a high number, early stopping will stop the training when the model is not improving (after patience epochs)
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3,
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weight,
)

Epoch 1/3


2025-09-18 18:26:44.786302: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
2025-09-18 18:26:46.175721: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91300


232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - f1_score: 0.6544 - loss: 1.1380

2025/09/18 18:27:06 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: cannot pickle 'module' object


232/232 ━━━━━━━━━━━━━━━━━━━━ 25s 75ms/step - f1_score: 0.7954 - loss: 0.7024 - val_f1_score: 0.9304 - val_loss: 0.2345 - learning_rate: 0.0010
Epoch 2/3
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - f1_score: 0.8892 - loss: 0.3746

2025/09/18 18:27:17 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: cannot pickle 'module' object


232/232 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - f1_score: 0.8971 - loss: 0.3543 - val_f1_score: 0.9401 - val_loss: 0.1997 - learning_rate: 0.0010
Epoch 3/3
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - f1_score: 0.9122 - loss: 0.2962

2025/09/18 18:27:28 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: cannot pickle 'module' object


232/232 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - f1_score: 0.9138 - loss: 0.2883 - val_f1_score: 0.9493 - val_loss: 0.1748 - learning_rate: 0.0010


In [11]:
from smart_r.utils import save_model_history


# Save the model history plot and the history as json to MLflow
save_model_history(history)

In [12]:
test_results = model.evaluate(test_dataset, return_dict=True)

# This will log all test metrics to MLflow with the prefix "test_" (like test_loss, test_accuracy, etc.)
for name, value in test_results.items():
    mlflow.log_metric(f"test_{name}", value)

47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - f1_score: 0.9346 - loss: 0.2055


In [13]:
from smart_r.utils import save_prediction_time

# Measure and log prediction time to MLflow and get the probabilities from the test dataset
y_probs = save_prediction_time(model, test_dataset)
y_pred = np.argmax(y_probs, axis=1)

# get the true labels from the test dataset
y_true = np.concatenate([y for x, y in test_dataset], axis=0)
y_true = np.argmax(y_true, axis=1)

2025-09-18 18:27:38.065475: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [14]:
from smart_r.utils import save_confusion_matrix, save_prediction_csv

file_paths = test_dataset.file_paths
class_names = test_dataset.class_names

save_confusion_matrix(y_true, y_pred, class_names)

save_prediction_csv(file_paths, y_true, y_pred, y_probs, class_names)